In [1]:
import matplotlib.pyplot as plt
import numpy.matlib
import numpy as np
import scipy
from scipy.cluster.hierarchy import DisjointSet
from scipy.stats import truncnorm
import math

from network0_def import *
from network1_config import *



In [3]:
n=5
k_mean = 2

A = np.array([
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 0, 0]
])
network = Network(adj_m=A)

In [4]:
A = network.adj_m
edge_ls = network.edge_list()

print(A)
print(edge_ls)
print(type(edge_ls))


[[0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 1 0 0]]
[[0 3]
 [1 3]
 [2 4]]
<class 'numpy.ndarray'>


In [5]:
lambda_ = 0.5
mask_states = np.zeros(len(edge_ls))
print(mask_states)

[0. 0. 0.]


In [6]:
w_s, w_i, w_b = 0.4, 0.6, 0.8

In [7]:
edge_ls_reverse = np.flip(edge_ls, axis=1)
edge_ls_double = np.concatenate((edge_ls, edge_ls_reverse), axis=0)
print(edge_ls_reverse)
print(edge_ls_double)

infect_edges = np.array([1,0,1,1,1,0])


[[3 0]
 [3 1]
 [4 2]]
[[0 3]
 [1 3]
 [2 4]
 [3 0]
 [3 1]
 [4 2]]


In [8]:
network_dir = Network_dir(n)
print(network_dir.adj_m)

[[False False False False False]
 [False False False False False]
 [False False False False False]
 [False False False False False]
 [False False False False False]]


In [9]:
# add each directed edge to network_dir based on the infection outcome
for idx, edge in enumerate(edge_ls_double):
    if infect_edges[idx] == 1:
        network_dir.add_edge(edge[0], edge[1])

In [10]:
print(network_dir.adj_m)
print(network_dir.neighbors_sink(0))
print(network_dir.neighbors_sink(1))
print(network_dir.neighbors_sink(2))
print(network_dir.neighbors_sink(3))
print(network_dir.neighbors_sink(4))

print(network_dir.adj_ls)

[[False False False  True False]
 [False False False False False]
 [False False False False  True]
 [ True  True False False False]
 [False False False False False]]
{3}
set()
{4}
{0, 1}
set()
[{3} set() {4} {0, 1} set()]


In [11]:
cluster_sizes = np.zeros(n)
for i in range(n):
    cluster_sizes[i] = network_dir.sink_size(i)

print(cluster_sizes)

[3. 1. 2. 3. 1.]


In [17]:
def SIR_djset_mask_test(n, edge_ls, lambda_, mask_states, w):
    """ For a single, fixed lambda value, run the SIR process on a network based on mask wearing 
    and return the average size of the final cluster.

    Assume single seed node to be any node and average final cluster size over all nodes."""

    w_s, w_i, w_b = w

    # create a double edge list which counts each edge twice in both directions.
    # switch the two columns of the edge list to get the reverse edge list.
    edge_ls_reverse = np.flip(edge_ls, axis=1)
    # concatenate the two edge lists to get the double edge list.
    edge_ls_double = np.concatenate((edge_ls, edge_ls_reverse), axis=0)

    # create a Network_dir object with no edges
    network_dir = Network_dir(n)



    # Assign weight value to each edge based on the mask wearing state of the source node 
    # (single direction only)
    w_edges = np.ones(edge_ls.shape[0])
    for idx, edge in enumerate(edge_ls):
        state_i = mask_states[edge[0]] # source node
        state_j = mask_states[edge[1]] # sink node

        if state_i == 0:
            if state_j == 0:
                pass
            else:
                w_edges[idx] = w_s # only susceptible sink node is wearing a mask
        else:
            if state_j == 0:
                w_edges[idx] = w_i # only infected source node is wearing a mask
            else:
                w_edges[idx] = w_b # both nodes are wearing masks

    # Account for both directions of the edge
    w_edges = np.concatenate((w_edges, swap_i_j(w_edges, w_s, w_i))) # shape (2*m,)

    # Weighted lambda value for each edge
    lambda_edges = lambda_ * w_edges

    # Randomly assign infection outcome to each edge based on lambda_edges (i.e., probability of weighted infection for each edge)
    # Flip the biased coin for each edge to determine if the edge is infected
    # infect_edges = np.random.binomial(1, lambda_edges)
    infect_edges = np.random.rand(len(lambda_edges)) < lambda_edges

    # add each directed edge to network_dir based on the infection outcome
    for idx, edge in enumerate(edge_ls_double):
        if infect_edges[idx] == 1:
            network_dir.add_edge(edge[0], edge[1])
        
    print("Edges infected: ", network_dir.edge_count())

    cluster_size = network_dir.sink_size(0) # assume the seed node is node 0
    return cluster_size
    
